# String simulator

In [2]:
from matplotlib import pyplot as plt
from matplotlib import animation as ani
import numpy as np
import scipy as sp

import time
import pickle

In [3]:
print("Last run: " + time.ctime())

Last run: Thu Aug  1 13:24:00 2024


## Setting up the solver

***
We're going to use Scipy's `integrate.solve_ivp` to solve our system of equations. To work with `solve_ivp`, we'll vectorize the system of $u$ and $v$, each of which is of length $n$, into a vector $p$, of length $2n$, such that $p_i=u_i$ for $1\leq i\leq n$, and $p_i=v_{i-n}$ for $n+1\leq i\leq 2n$:

$$p=\left[u_1, u_2, u_3, \dots u_n, v_1, v_2, v_3, \dots v_n\right]$$

Its derivative (using dot notation for $\frac{d}{dt}$) is:

$$\dot{p}=\left[\dot{u}_1, \dot{u}_2, \dot{u}_3, \dots \dot{u}_n, \dot{v}_1, \dot{v}_2, \dot{v}_3, \dots \dot{v}_n\right]$$

We will relate $p$ and $\dot{p}$ with a matrix of coefficients, $C$:

$$\dot{p}=pC$$

The coefficient $c_{ij}$ in $C$ should relate $p_i$ with $\dot{p}_j$:

$$\dot{p}_j=\sum_{i}p_ic_{ij}$$

Substituting $u_i$ with $p_i$ and $v_i$ with $p_{i+n}$:

$$p_{i+n}=\frac{dp_i}{dt}$$

$$\frac{dp_{i+n}}{dt}=\frac{T}{\rho}\frac{p_{i+1}+p_{i-1}-2p_i}{\Delta x^2}-\mu_i p_{i+n}$$

We can determine the coefficients $c_{ij}$ by examining the above equations:

$$c_{ij}=\begin{cases}
    0&\text{for }j=n+1\text{ or }j=2n\\
    1&\text{for }i=j+n\\
    \frac{T}{\Delta x^2\rho}&\text{for }j=i+n-1\text{ or }j=i+n+1\\
    \frac{-2T}{\Delta x^2\rho}&\text{for }j=i+n\\
    -\mu_{i-n}&\text{for }i=j\text{ and }i>n
\end{cases}$$
This gives a system of $2n$ first-order ODEs which we can solve with `solve_ivp`.

***
The following values for tension, linear density, and length are approximately the conditions of the E string on my bass guitar. I measured the length, looked up the tension of a certain string from a certain manufacturer (a 0.105" nickel-wound roundwound string from a manufacturer whose name rhymes with mario), and calculated the linear density that would be needed to produce a fundamental frequency of E0.

In [8]:
T = 131.6
r = 0.033
L = 0.762
#fundamental frequency according to mersenne's laws:
print("f0 = {0:.2f}".format(np.sqrt(T/r)/L/2))

f0 = 41.44


This is the function we'll use to simulate the string. We're going to keep $n=500$ just to keep things simple.

In [4]:
def simulate(tension, density, length, damping, initial, n_points=500):
    """simulate a string given length, tension, linear density
    All quantities should be given in SI units -
    tension should have units of N,
    density should have units of kg/m,
    length should have units of m,
    time should have units of seconds
    """

    if damping.shape[0] != n_points:
        raise ValueError("Damping vector must have length n_points")
    elif initial.shape[0] != 2*n_points:
        raise ValueError("Initial condition vector must have length 2*n_points")
    
    c = np.zeros((2*n_points, 2*n_points))

    dx = length/(n_points-1)

    st = time.time()
    print("Beginning...")

    #create c matrix according to rules above
    for i in range(0, 2*n_points):
        for j in range(0, 2*n_points):
            if j==n_points or j==2*n_points-1:
                c[i,j] = 0
            elif i == j+n_points:
                c[i,j]=1
            elif j==i+n_points-1 or j==i+n_points+1:
                c[i,j]=tension/(dx**2*density)
            elif j == i+n_points:
                c[i,j]=-2*tension/(dx**2*density)
            elif i==j and i > n_points:
                c[i,j] = -damping[i-n_points]

    print("Created c matrix in {0:.2f} s".format(time.time()-st))
          
    def func(t, y):
        return np.matmul(y, c)

    #48000 hz sampling frequency
    t_eval = np.linspace(0, 1, 48000)

    #way faster with 3(2) than with 5(4)
    s = sp.integrate.solve_ivp(func, (0,1), initial, t_eval=t_eval, method="RK23")
    
    print("Solver finished in {0:.2f} s".format(time.time()-st))
          
    return s

These are the initial conditions we'll use. `p0_p` represents the string plucked near the bridge. `p0_n` represents the string plucked closer to the neck. `p0_h` represents the string struck near the bridge.

In [5]:
p0_p = np.zeros(1000)
p0_n = np.zeros(1000)
p0_h = np.zeros(1000)

#initial displacement of 0.03 m max
p0_p[0:450]=[0.03*i/450 for i in range(0,450)]
p0_p[450:499]=[0.03*(49-i)/49 for i in range(0,49)]

p0_n[0:300]=[0.03*i/300 for i in range(0,300)]
p0_n[300:499]=[0.03*(199-i)/199 for i in range(0,199)]

#initial velocity of 15 m/s max
p0_h[945:955] = [3, 6, 9, 12, 15, 15, 12, 9, 6, 3]

These are the damping profiles we'll use. `mu_open` is a light uniform damping across the entire string. `mu_palm` is a region of higher damping near the bridge, representing a palm mute. `mu_har2` and `mu_har3` have points of high damping at $x=\frac{L}{2}$ and $x=\frac{L}{3}$, respectively.

In [6]:
mu_open = np.full(500, 0.5)

mu_palm = np.full(500, 0.5)
mu_palm[-50:] = 200

mu_har2 = np.full(500, 0.5)
mu_har2[249:251] = 5000

mu_har3 = np.full(500, 0.5)
mu_har3[166:168] = 5000

Here we'll actually run the simulations:

In [7]:
s_op_p = simulate(T, r, L, mu_open, p0_p) #open string, plucked near the pickup
s_op_n = simulate(T, r, L, mu_open, p0_n) #open string, plucked near the neck
s_op_h = simulate(T, r, L, mu_open, p0_h) #open string, hammered
s_palm = simulate(T, r, L, mu_palm, p0_p) #palm muted string, plucked near the pickup
s_har2 = simulate(T, r, L, mu_har2, p0_p) #string muted for 2nd harmonic
s_har3 = simulate(T, r, L, mu_har3, p0_p) #string muted for 3rd harmonic

Beginning...
Created c matrix in 0.41 s
Solver finished in 44.96 s
Beginning...
Created c matrix in 0.35 s
Solver finished in 37.58 s
Beginning...
Created c matrix in 0.31 s
Solver finished in 35.19 s
Beginning...
Created c matrix in 0.33 s
Solver finished in 36.75 s
Beginning...
Created c matrix in 0.33 s
Solver finished in 37.49 s
Beginning...
Created c matrix in 0.29 s
Solver finished in 38.48 s


And save them to `.dat` files. Further analysis will be done in `analysis.ipynb`.

In [8]:
with open("op_p.dat", "wb") as f:
    pickle.dump(s_op_p, f)

with open("op_n.dat", "wb") as f:
    pickle.dump(s_op_n, f)

with open("op_h.dat", "wb") as f:
    pickle.dump(s_op_h, f)

with open("palm.dat", "wb") as f:
    pickle.dump(s_palm, f)

with open("har2.dat", "wb") as f:
    pickle.dump(s_har2, f)

with open("har3.dat", "wb") as f:
    pickle.dump(s_har3, f)